# AI-LAB SESSION 1: Uninformed search

In this first session we will work on uninformed search

## Maze environments

The environments used are **SmallMaze** (visible in the figure) and its variations
![SmallMaze](images/maze.png)
The agent starts in cell $(0, 2)$ and has to reach the treasure in $(4, 3)$

## Assignment 1

Your first assignment is to implement the BFS algorithm on **SmallMaze**. In particular, you are required to implement both *tree_search* and *graph_search* versions of BFS that will be called by the generic *bfs*.

The results returned by your *bfs* must be a tuple $(path, stats)$ in the following form:
* *path* - tuple of state identifiers forming a path from the start state to the goal state. ``None`` if no solution is found
* *stats* - tuple of:
     * *time* - time elapsed between the start and the end of the algorithm
     * *expc* - number of nodes explored. A node is considered as explored when removed from the fringe and analyzed
     * *maxnodes* - maximum number of nodes in memory at the same time (fringe + closed)

After the correctness of your implementations have been assessed, you can run the algorithms on other two maze environments: **GrdMaze** and **BlockedMaze**.

Function *build_path* can be used to return a tuple of states from the root node to another node by following *parent* links

In [15]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import gym
import envs
from timeit import default_timer as timer
from utils.fringe import FringeNode, QueueFringe

def build_path(node):
    """
    Builds a path going backward from a node
    
    Args:
        node: node to start from
        
    Returns:
        path from root to ``node``
    """
    path = []
    while node.parent is not None:
        path.append(node.state)
        node = node.parent
    return tuple(reversed(path))

The next two functions have to be implemented

In [25]:
def tree_search(env, fringe):
    """
    Tree search
    
    Args:
        environment: OpenAI Gym environment
        fringe: instance of Fringe data structure
        
    Returns:
        (path, stats): solution as a path and stats.
        The stats are a tuple of (expc, maxnodes): number of explored states, max nodes in memory
    """
    
    #VI E' UNA SORTA DI INNER [[[POSITION] STATE] NODE]
    
    start = env.startstate
    goal = env.goalstate
    explored_states = 0
    
    fringe.add(FringeNode(start))
    
    while not fringe.is_empty():
        currentNode = fringe.remove()
        explored_states += 1
        if(currentNode.state == goal): #?? è giusto fare il controllo in questa posizione? 
                                        ## o devo farlo quando faccio lo scan dei nextNodes di un nodo
            return build_path(currentNode), (explored_states, len(fringe) + 1)
        for action in range(env.action_space.n):
            nextNode = FringeNode(env.sample(currentNode.state, action), currentNode)
            #if(nextNode.state != currentNode.state):
            fringe.add(nextNode)
    
    return None, (explored_states, len(fringe) + 1) #+1 è il nodo che ho esaminato

    """
        explored_node=0;
        if environment.startstate == environment.goalstate :
            return [FringeNode(environment.startstate)],(1,1)
        fringe.add(FringeNode(environment.startstate))
        while True :
            if fringe.is_empty():
                return None,(explored_node,len(fringe))
            else:
                node=fringe.remove()
                explored_node=explored_node+1
                for action in range (environment.action_space.n):
                    child = FringeNode(environment.sample(node.state,action),node)
                    if node.state == environment.goalstate :
                        return build_path(node), (explored_node,len(fringe)+1)
                    fringe.add(child)
    """

####### TEST ########
#environment = gym.make("SmallMaze-v0")
#tree_search(environment, QueueFringe());

In [38]:
def graph_search(environment, fringe):
    """
    Graph search
    
    Args:
        environment: OpenAI Gym environment
        fringe: instance of Fringe data structure
        
    Returns:
        (path, stats): solution as a path and stats.
        The stats are a tuple of (expc, maxnodes): number of explored nodes, max nodes in memory
    """
    
    closed = set()
    start = env.startstate
    goal = env.goalstate
    Nexploredstates = 0
    
    fringe.add(FringeNode(start))
    
    while not fringe.is_empty():
        currentNode = fringe.remove()
        if(currentNode.state not in closed):
            closed.add(currentNode.state)
        Nexploredstates += 1
        if(currentNode.state == goal): #?? è giusto fare il controllo in questa posizione? 
                                        ## o devo farlo quando faccio lo scan dei nextNodes di un nodo
            return build_path(currentNode), (Nexploredstates, len(closed))
        for action in range(env.action_space.n):
            nextNode = FringeNode(env.sample(currentNode.state, action), currentNode)
            if(nextNode.state not in closed): #nextNode.state not in fringe serve qui? non è necessario
                closed.add(nextNode.state)
                fringe.add(nextNode)
    
    return None, (Nexploredstates, len(closed))

In [27]:
def bfs(environment, search_type):
    """
    Breadth-first search
    
    Args:
        environment: OpenAI Gym environment
        search_type: type of search - tree_search or graph_search (function pointer)
        
    Returns:
        (path, stats): solution as a path and stats.
        The stats are a tuple of (time, expc, maxnodes): elapsed time, number of explored nodes, max nodes in memory
    """
    t = timer()
    path, stats = search_type(environment, QueueFringe())
    return path, (timer() - t, stats[0], stats[1])

The following code calls your tree search version of BFS and prints the results

In [23]:
envname = "SmallMaze-v0"  # Other options are GrdMaze-v0 and BlockedMaze-v0

print("\n----------------------------------------------------------------")
print("\tTREE SEARCH")
print("\tEnvironment: ", envname)
print("----------------------------------------------------------------\n")

# Create and render the environment
env = gym.make(envname)

# Create and render the environment
env = gym.make(envname)
env.render()
solution, stats = bfs(env, tree_search)  # Perform BFS
if solution is not None:
    solution = [env.state_to_pos(s) for s in solution]
    
# Print stats and path
print("\n\nExecution time: {0}s\nN° of nodes explored: {1}\nMax n° of nodes in memory: {2}\nSolution: {3}".format(
        round(stats[0], 4), stats[1], stats[2], solution))


----------------------------------------------------------------
	TREE SEARCH
	Environment:  GrdMaze-v0
----------------------------------------------------------------

[['C' 'C' 'C' 'S']
 ['C' 'C' 'W' 'C']
 ['C' 'C' 'C' 'C']
 ['C' 'W' 'W' 'W']
 ['C' 'C' 'C' 'G']]


Execution time: 37.1345s
N° of nodes explored: 365867
Max n° of nodes in memory: 1097599
Solution: [(0, 2), (0, 1), (0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3)]


Correct results for BFS tree search can be found [here](files/results/bfs_tree_search_results.txt)

The following code calls your graph search version of BFS and prints the results

In [41]:
envname = "SmallMaze-v0"  # Other options are GrdMaze-v0 and BlockedMaze-v0

print("\n----------------------------------------------------------------")
print("\tGRAPH SEARCH")
print("\tEnvironment: ", envname)
print("----------------------------------------------------------------\n")

# Create and render the environment
env = gym.make(envname)
env.render()
solution, stats = bfs(env, graph_search)  # Perform BFS
if solution is not None:
    solution = [env.state_to_pos(s) for s in solution]
    
# Print stats and path
print("\n\nExecution time: {0}s\nN° of nodes explored: {1}\nMax n° of nodes in memory: {2}\nSolution: {3}".format(
        round(stats[0], 4), stats[1], stats[2], solution))


----------------------------------------------------------------
	GRAPH SEARCH
	Environment:  BlockedMaze-v0
----------------------------------------------------------------

[['C' 'C' 'S' 'C']
 ['C' 'C' 'W' 'C']
 ['C' 'C' 'C' 'C']
 ['C' 'C' 'W' 'W']
 ['C' 'C' 'W' 'G']]


Execution time: 0.0044s
N° of nodes explored: 15
Max n° of nodes in memory: 15
Solution: None


Correct results for BFS graph search can be found [here](files/results/bfs_graph_search_results.txt)

## Assignment 2

Your second assignment is to implement the IDS algorithm on **SmallMaze**. In particular, you are required to implement both *dls_ts* (depth-limited tree search) and *dls_gs* (depth-limited graph search) versions of IDS that will be called by the generic *ids*. The recursions must be implemented in *rdls_ts* (recursive depth-limited tree search) and *rdls_gs* (recursive depth-limited graph search) called by *dls_ts* and *dls_gs* respectively.

Similarly to assignment 1, the results returned by your *ids* must be a tuple $(path, stats)$ in the following form:
* *path* - tuple of state identifiers forming a path from the start state to the goal state. ``None`` if no solution is found
* *stats* - tuple of:
     * *time* - time elapsed between the start and the end of the algorithm
     * *expc* - number of nodes explored. A node is considered as explored when removed from the fringe and analyzed
     * *maxnodes* - maximum number of nodes in memory at the same time (the depth of the recursion stack + closed)

After the correctness of your implementations have been assessed, you can run the algorithms on other two maze environments: **GrdMaze** and **BlockedMaze**.

**FringeNode** has a useful variable that can be set in the constructor and can be used to track the depth of a node in the path (and consequently of the recursion stack of IDS): *pathcost*. If the root node has a *pathcost* of 0, its children will have a *pathcost* increased by 1

In [42]:
start = env.startstate
root = FringeNode(start)  # parent = None and pathcost = 0 as default
child = FringeNode(env.sample(start, 0), root, root.pathcost + 1)  # pathcost is the third argument
print("Root pathcost: {}\tChild pathcost: {}".format(root.pathcost, child.pathcost))

Root pathcost: 0	Child pathcost: 1


Here you can implement the various functions requested

In [61]:
def dls_ts(environment, limit):
    """
    Depth-limited search (tree search)
    
    Args:
        environment: OpenAI Gym environment
        limit: depth limit budget
        
    Returns:
        (path, cutoff, stats): solution as a path, cutoff flag and stats.
        The stats are a tuple of (time, expc, maxnodes): elapsed time, number of explored nodes, max nodes in memory
    """
    t = timer()
    path, cutoff, expc, maxdepth = rdls_ts(environment, FringeNode(environment.startstate), limit)
    return path, cutoff, (timer() - t, expc, maxdepth)

In [44]:
def rdls_ts(env, node, limit):
    """
    Recursive depth-limited search (tree search version)
    
    Args:
        environment: OpenAI Gym environment
        node: node to explore
        limit: depth limit budget
    
    Returns:
        (path, cutoff, expc, maxdepth): path, cutoff flag, number of explored nodes, max nodes in memory
    """
    """
    function Recursive-DLS(node, problem, limit)
        if problem.Goal-Test(node.State) then return Solution(node)
        if limit = 0 then return Cutoff
        cutof f occurred ← F alse
        for each action in problem.Actions(node.State) do
            child ← Child-Node(problem, node, action)
            result ← Recursive-DLS(child, problem, limit − 1)
            if result = Cutoff then cutof f occurred ← T rue
            else if result =
            Failure then return result
        if cutof f occurred then return Cutoff
        return Failure
    """
    if(node.state == env.goalstate):
        return (build_path(node), False, 1, limit)
    if(limit == 0):
        return (None, True, 1, limit)
    expcA = 0
    for action in range(env.action_space.n):
        childNode = FringeNode(env.sample(node.state, action), node)
        result, cutoff, expc, maxdepth = rdls_ts(env, childNode, limit - 1)
        expcA += expc
        if(not cutoff):
            return (result, False, expcA + 1, limit)
    return None, True, expcA + 1, limit

In [72]:
def dls_gs(environment, limit):
    """
    Depth-limited search (graph search)
    
    Args:
        environment: OpenAI Gym environment
        limit: depth limit budget
        
    Returns:
        (path, cutoff, stats): solution as a path, cutoff flag and stats.
        The stats are a tuple of (time, expc, maxnodes): elapsed time, number of explored nodes, max nodes in memory
    """
    t = timer()
    path, cutoff, expc, maxdepth = rdls_gs(environment, FringeNode(environment.startstate), limit, set())
    return path, cutoff, (timer() - t, expc, maxdepth)

In [83]:
def rdls_gs(environment, node, limit, closed):
    """
    Recursive depth-limited search (graph search version)
    
    Args:
        environment: OpenAI Gym environment
        node: node to explore
        limit: depth limit budget
        closed: completely explored nodes
        
    Returns:
        (path, cutoff, expc, maxdepth): path, cutoff flag, n° of nodes explored, max nodes in memory
    """
    
    """
    closed = set()
    start = env.startstate
    goal = env.goalstate
    Nexploredstates = 0
    
    fringe.add(FringeNode(start))
    
    while not fringe.is_empty():
        currentNode = fringe.remove()
        if(currentNode.state not in closed):
            closed.add(currentNode.state)
        Nexploredstates += 1
        if(currentNode.state == goal): #?? è giusto fare il controllo in questa posizione? 
                                        ## o devo farlo quando faccio lo scan dei nextNodes di un nodo
            return build_path(currentNode), (Nexploredstates, len(closed))
        for action in range(env.action_space.n):
            nextNode = FringeNode(env.sample(currentNode.state, action), currentNode)
            if(nextNode.state not in closed): #nextNode.state not in fringe serve qui? non è necessario
                closed.add(nextNode.state)
                fringe.add(nextNode)
    
    return None, (Nexploredstates, len(closed))
    """
    
    if(node.state not in closed):
        closed.add(node.state)
    if(node.state == env.goalstate):
        return (build_path(node), False, 1, limit)
    if(limit == 0):
        return (None, True, 1, limit)
    expcA = 0
    for action in range(env.action_space.n):
        childNode = FringeNode(env.sample(node.state, action), node)
        if(childNode.state not in closed):
            result, cutoff, expc, maxdepth = rdls_gs(env, childNode, limit - 1, closed)
            expcA += expc
            if(not cutoff):
                return (result, False, expcA + 1, limit)
    return None, True, expcA + 1, limit

In [74]:
def ids(environment, search_type):
    """
    Iterative deepening depth-first search
    
    Args:
        environment: OpenAI Gym environment
        search_type: type of search (graph or tree) - dls_gs or dls_ts (function pointer)
    
    Returns:
        (path, stats): solution as a path and stats.
        The stats are a tuple of (time, expc, maxnodes): elapsed time, number of explored nodes, max nodes in memory
    # Pseudocode
    # from depth = 0 to infinity
    #   solution, cutoff, stats = stype(problem, depth)
    """
    t = timer()
    cutoff = True
    depth = 0
    expc = 0
    time = 0
    min_limit = 99999999999999999999999999999
    
    while(cutoff == True):
        solution, cutoff, stats = search_type(environment, depth)
        #AGGIORNO STATS
        expc += stats[1]
        time = stats[0]
        if (min_limit > stats[2]):
            min_limit = stats[2]
        depth +=1
    return solution, (time, expc, depth - min_limit)

The following code calls your tree search version of IDS and prints the results

In [84]:
envname = "SmallMaze-v0"  # Other options are GrdMaze-v0 and BlockedMaze-v0

print("\n----------------------------------------------------------------")
print("\tTREE SEARCH")
print("\tEnvironment: ", envname)
print("----------------------------------------------------------------\n")

# Create and render the environment
env = gym.make(envname)
env.render()
solution, stats = ids(env, dls_ts)  # Perform BFS
if solution is not None:
    solution = [env.state_to_pos(s) for s in solution]
    
# Print stats and path
print("\n\nExecution time: {0}s\nN° of nodes explored: {1}\nMax n° of nodes in memory: {2}\nSolution: {3}".format(
        round(stats[0], 4), stats[1], stats[2], solution))


----------------------------------------------------------------
	TREE SEARCH
	Environment:  SmallMaze-v0
----------------------------------------------------------------

[['C' 'C' 'S' 'C']
 ['C' 'C' 'W' 'C']
 ['C' 'C' 'C' 'C']
 ['C' 'W' 'W' 'W']
 ['C' 'C' 'C' 'G']]


Execution time: 0.5248s
N° of nodes explored: 138298
Max n° of nodes in memory: 10
Solution: [(0, 1), (0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3)]


Correct results for IDS tree search can be found [here](files/results/ids_tree_search_results.txt)

The following code calls your graph search version of BFS and prints the results

In [87]:
envname = "SmallMaze-v0"  # Other options are GrdMaze-v0 and BlockedMaze-v0

print("\n----------------------------------------------------------------")
print("\tGRAPH SEARCH")
print("\tEnvironment: ", envname)
print("----------------------------------------------------------------\n")

# Create and render the environment
env = gym.make(envname)
env.render()
solution, stats = ids(env, dls_gs)  # Perform BFS
if solution is not None:
    solution = [env.state_to_pos(s) for s in solution]
    
# Print stats and path
print("\n\nExecution time: {0}s\nN° of nodes explored: {1}\nMax n° of nodes in memory: {2}\nSolution: {3}".format(
        round(stats[0], 4), stats[1], stats[2], solution))


----------------------------------------------------------------
	GRAPH SEARCH
	Environment:  GrdMaze-v0
----------------------------------------------------------------

[['C' 'C' 'C' 'S']
 ['C' 'C' 'W' 'C']
 ['C' 'C' 'C' 'C']
 ['C' 'W' 'W' 'W']
 ['C' 'C' 'C' 'G']]


Execution time: 0.0008s
N° of nodes explored: 128
Max n° of nodes in memory: 13
Solution: [(0, 2), (0, 1), (0, 0), (1, 0), (1, 1), (2, 1), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3)]


Correct results for IDS graph search can be found [here](files/results/ids_graph_search_results.txt)

## Discussion

Now that you have correctly implemented both BFS and IDS what can you say about the solutions they compute? Are there significant differences in the stats?